# AWS S3 Buckets: Uploading and Downloading

AWS S3 buckets provide scalable, secure, reliable data storage in the AWS cloud.

 - **This notebook will throw some errors for demonstration purposes. Do not be alarmed.**

AWS provides software to interact with the S3 API:
  - [AWS CLI](https://aws.amazon.com/cli/) for access from the command line
    - There are 2 major version of awscli, v1 and v2, which use different authentication paradigms.
      - This notebook covers awscli v1
      - awscli v2 content will be added at a later date
  - [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) for programmatic Python access

S3 Buckets may be **public** or **private**

---

## **1 Downloading from Public S3 Buckets**


### **1.1 Downloading from the command line with `AWS CLI`**
- does not require authentication
- if not authenticating, requires use of the `--no-sign-request` argument

The `aws s3 cp` command works for downloading or uploading from/to S3:

`aws s3 cp source_path destination_path`

---

#### **1.1.1 Attempt downloading an example file from a public S3 bucket without using the `--no-sign-request` argument**

- This will cause an error unless you have previously configured the aws-cli with your AWS credentials

In [ ]:
!aws s3 cp s3://asf-jupyter-data-west/S3_example/example.txt example.txt

#### **1.1.2 Run the `aws s3 cp` command again, adding the `--no-sign-request` argument**

- Look for the downloaded file in the file browser

In [ ]:
!aws s3 cp --no-sign-request s3://asf-jupyter-data-west/S3_example/example.txt example.txt

---

### **1.2 Downloading in Python with `boto3`**

You can build S3 access into your Python scripts and automated workflows with `boto3`

#### **1.2.1 To prepare for the following steps, delete the local copy of `example.txt`, which we previously downloaded**

In [ ]:
from pathlib import Path

example_path = Path.cwd()/"example.txt"
example_path.unlink(missing_ok=True)

#### **1.2.2 Run the following code cell to attempt downloading `example.txt` with `boto3` from a public S3 bucket without declaring an unsigned signature or providing credentials** 

In [ ]:
import boto3

s3 = boto3.resource('s3')

bucket_name = "asf-jupyter-data-west"
bucket = s3.Bucket(bucket_name)

# objects in S3 bucket storage are called keys
example_key = "S3_example/example.txt"

bucket.download_file(example_key, example_path) 

That didn't work; we received a `NoCredentialsError`. We need to provide our AWS credentials or declare ourselves as anonymous users.

---
#### **1.2.2 Try this again, but declare yourself anonymous by providing the config signature version: `UNSIGNED`**

It may take a few moments for the file to appear in the file browser. You can hit the file browser's refresh button to make it appear sooner.

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

bucket_name = "asf-jupyter-data-west"
bucket = s3.Bucket(bucket_name)

bucket.download_file(example_key, example_path) 

---

## **2 Configuring awscli v1 to add your AWS credentials**

- You will need an `AWS Access Key ID` and `AWS Secret Access Key` from an AWS IAM Role with permissions to access any needed private S3 Buckets
  - https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_access-keys.html#Using_CreateAccessKey

These credentials are discoverable by both `aws-cli` and `boto3` and will allow you to:
- upload files to public S3 buckets if you have an access key for the account and your IAM user has permissions
- access private buckets if you have an access key for the account and your IAM user has permissions

---

#### **2.1 Open a launcher, Drag this notebook tab into split-screen mode, and open a terminal:**

<img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/workshops/nisar_early_adopters/open_terminal.gif" width=75%/>

---

#### **2.2 Configure your default AWS profile**

- Enter `aws configure` in the terminal
  - Enter your AWS Access Key ID
  - Enter your AWS Secret Access Key
  - Enter a region ("us-west-2" for ASF data access)
  - Enter "json" as an output format

<img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/workshops/nisar_early_adopters/aws_creds.gif" width=75%/>

---

#### **2.3 Configure a non-default AWS profile**

Adding profiles will allow you to access buckets in different AWS accounts using different sets of credentials

**Repeat the steps in 1.3.2, changing the following:**
- change `aws configure` to  `aws configure --profile your_profile_name`
    - replace `your_profile_name` with your chosen profile name
    
This will produce a `~/.aws/credentials` file that looks something like this:

```
[default]
aws_access_key_id = key_id_for_account_1
aws_secret_access_key = access_key_for_account_1

[profile_name_for_account_2]
aws_access_key_id = key_id_for_account_2
aws_secret_access_key = access_key_for_account_2
```

---
## **3 Uploading to S3 Buckets with Credentials**

### **3.1 Uploading with `awscli`**

#### **3.1.1 Swapping the source and destination paths in our previous awscli download command will upload the file to the same location**

- Even though the bucket is public, you will not be able to write to it unless the IAM user owning your Access Key has permission
- uploading the file will overwrite the copy in the S3 bucket

**The command below uses your default AWS profile**

In [ ]:
!aws s3 cp example.txt s3://asf-jupyter-data-west/S3_example/example.txt 

---
#### **3.1.2 If you added your credentials under a profile, you will need to use the `--profile` argument and provide your profile name**

In [ ]:
!aws s3 --profile osl cp example.txt s3://asf-jupyter-data-west/S3_example/example.txt 

---
### **3.2 Uploading with `boto3`**

#### **3.2.1 Upload `example.txt` with `boto3`**

- if your default AWS profile is not configured to access the bucket, the next command will fail (this is okay)

In [ ]:
import boto3

s3 = boto3.resource('s3')

bucket_name = "asf-jupyter-data-west"
bucket = s3.Bucket(bucket_name)

# objects in S3 bucket storage are called keys
example_key = "S3_example/example.txt" 

bucket.upload_file(example_path, example_key)

---
#### **3.2.2 If you added your credentials under a profile, you will need instantiate a `boto3` session with your profile name**
- `session = boto3.Session(profile_name='your _profile_name')`

In [ ]:
s3 = boto3.resource('s3')

bucket_name = "asf-jupyter-data-west"
session = boto3.Session(profile_name='osl')
s3 = session.client('s3')

# objects in S3 bucket storage are called keys
example_key = "S3_example/example.txt" 

s3.upload_file(example_path, bucket_name, example_key)

